In [9]:
# Install dependencies
!pip install librosa soundfile ipywidgets pydub

import librosa
import numpy as np
import soundfile as sf
from google.colab import files
import ipywidgets as widgets
from IPython.display import Audio, display


In [17]:
def detect_gender(y, sr):
    # Estimate pitch contour
    f0 = librosa.yin(y, fmin=50, fmax=300, sr=sr)
    mean_f0 = np.nanmean(f0)

    if mean_f0 < 165:
        return "male", mean_f0
    else:
        return "female", mean_f0


In [18]:

# ==============================
# Voice Effect Functions
# ==============================
def robotic_voice(y, sr):
    # Ring modulation (classic robot effect)
    t = np.linspace(0, len(y)/sr, len(y))
    modulator = np.sin(2*np.pi*30*t)  # 30Hz modulation
    robotic = y * modulator

    # Flatten pitch with STFT
    D = librosa.stft(robotic)
    mag, phase = np.abs(D), np.angle(D)
    robotic = librosa.istft(mag * np.exp(1j * np.sign(phase)))

    # Slight distortion for metallic effect
    robotic = np.tanh(robotic * 3)
    return robotic


def male_voice(y, sr):
    gender, f0 = detect_gender(y, sr)
    if gender == "male":
        # already male → shift slightly lower
        return librosa.effects.pitch_shift(y=y, sr=sr, n_steps=-2)
    else:
        # female to male → bigger shift down
        return librosa.effects.pitch_shift(y=y, sr=sr, n_steps=-5)

def female_voice(y, sr):
    gender, f0 = detect_gender(y, sr)
    if gender == "female":
        # already female → slight brightening
        return librosa.effects.pitch_shift(y=y, sr=sr, n_steps=2)
    else:
        # male to female → bigger shift up
        return librosa.effects.pitch_shift(y=y, sr=sr, n_steps=5)

def baby_voice(y, sr):
    gender, f0 = detect_gender(y, sr)
    if gender == "female":
        shift = 5  # less shift (otherwise becomes chipmunk)
    else:
        shift = 7  # more shift for male voice
    y = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=shift)
    y = librosa.effects.time_stretch(y=y, rate=1.2)
    return y



In [19]:

# ==============================
# Upload Audio
# ==============================
print("Upload an audio file (WAV/MP3 recommended):")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
y, sr = librosa.load(filename, sr=None)
print(f"Loaded {filename} - {len(y)/sr:.2f} seconds, Sample rate = {sr}")


Upload an audio file (WAV/MP3 recommended):


Saving Recording (2).mp3 to Recording (2) (2).mp3
Loaded Recording (2) (2).mp3 - 6.86 seconds, Sample rate = 48000


In [20]:

# ==============================
# Select Effect
# ==============================
effect_dropdown = widgets.Dropdown(
    options=["robotic", "male", "female", "baby"],
    description="Effect:"
)
display(effect_dropdown)

button = widgets.Button(description="Apply Effect")

output_widget = widgets.Output()

def on_button_click(b):
    with output_widget:
        output_widget.clear_output()
        effect = effect_dropdown.value
        if effect == "robotic":
            processed = robotic_voice(y, sr)
        elif effect == "male":
            processed = male_voice(y, sr)
        elif effect == "female":
            processed = female_voice(y, sr)
        elif effect == "baby":
            processed = baby_voice(y, sr)

        # Save processed file
        out_file = f"processed_{effect}.wav"
        sf.write(out_file, processed, sr)

        # Play and download
        print(f"Effect applied: {effect}")
        display(Audio(out_file, autoplay=True))
        files.download(out_file)

button.on_click(on_button_click)
display(button, output_widget)


Dropdown(description='Effect:', options=('robotic', 'male', 'female', 'baby'), value='robotic')

Button(description='Apply Effect', style=ButtonStyle())

Output()